## 0. Install and Import Dependencies

In [ ]:
!pip install easyocr
!pip install imutils

: 

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

: 

## 1. Read in Image, Grayscale and Blur

In [ ]:
img = cv2.imread('/content/image4.jpg')

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

: 

: 

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

: 

## 2. Apply filter and find edges for localization

In [ ]:
bfilter = cv2.bilateralFilter(gray, 11, 50, 50) #Noise reduction

plt.imshow(bfilter, cmap='gray')

: 

In [ ]:
edged = cv2.Canny(bfilter, 5, 250) #Edge detection
plt.imshow(edged, cmap='gray')

: 

## 3. Find Contours and Apply Mask

In [ ]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

: 

In [ ]:
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

: 

In [ ]:
for contour in contours:
  approx = cv2.approxPolyDP(contour, 10, True)
  print(approx)
  print("----------------")
  print("\n")

: 

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
plt.imshow(mask, cmap='gray')


: 

In [ ]:
new_image = cv2.drawContours(mask, [location], 0,255, -1)
plt.imshow(new_image, cmap='gray')


: 

In [ ]:
new_image = cv2.bitwise_and(img, img, mask=mask)
plt.imshow(new_image, cmap='gray')

: 

In [ ]:
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

: 

In [ ]:
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = gray[x1:x2+1, y1:y2+1]

: 

In [ ]:
(x,y) = np.where(mask==255)
(x,y)

: 

In [ ]:
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

: 

## 4. Use Easy OCR To Read Text

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
result

: 

## 5. Render Result

In [ ]:
text = result[0][-2]
font = cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

: 

: 

: 